## Софья Скобелева
## Практическое задание №3 

Импортируем библиотеки:

In [1]:
import nltk
from nltk.corpus import stopwords # список "стоп-слов"
from nltk.tokenize import word_tokenize # для токенизации
from nltk.stem import WordNetLemmatizer # для лемматизации
from pymorphy2 import MorphAnalyzer # для проведения морфологического анализа
from nltk.util import ngrams # для создания n-грамм
from collections import Counter

import pandas as pd
import numpy as np
import string

import gensim # для тематического моделирования
from gensim import corpora

from sklearn.feature_extraction.text import TfidfVectorizer # для применения TF-IDF
from gensim.models import Word2Vec # для поиска близких по смыслу слов

Загрузим три песни на русском языке:

In [2]:
song1 = open('song1.txt', 'r').read()
song2 = open('song2.txt', 'r').read()
song3 = open('song3.txt', 'r').read()

Посмотрим как они выглядят:

In [3]:
song1

'Есть такие дороги — назад не ведут\nНа чужом берегу я прилив стерегу.\nПаруса обманув, ветер стих навсегда,\nПлоским зеркалом стала морская вода.\nОбернуться бы лентой в чужих волосах.\nПлыть к тебе до рассвета, не ведая страх,\nШёлком в руки родные опуститься легко.\nВспоминай моё имя, прикасайся рукой.\nЯ по дну бы морскому навстречу пошла,\nТолько в компасе старом сломалась игла.\nПарус стёрся до дыр от палящих светил\nДа и ветер попутный меня не взлюбил.\nВетер, брат ты мой, ветер, за что осерчал?\nХороню в себе боль и венчаю печаль.\nБурунами морскими пробежать нелегко\nВспоминай моё имя, прикасайся рукой.\nТретий год я зову, только эхо в ответ,\nОбманул меня ветер, запутал твой след.\nТолько сталь твоих глаз не забыть никогда,\nА в груди ледяная морская вода.\nОбернуться бы лентой в чужих волосах.\nПлыть к тебе до рассвета, не ведая страх,\nШёлком в руки родные опуститься легко.\nВспоминай моё имя...\nВетер, брат ты мой, ветер, за что осерчал?\nХороню в себе боль и венчаю печаль

In [4]:
song2

'А судьба выбирает несмелых \nИ с нею порой не везло \nХоть бы раз прожить по размеру \nНо мне пока всё велико Веди в это дивное место \nГде из детей вываривают детство \nГоворят, там храбрейших обитель \nСердца их покрыла броня \nКто дойдет, тому выдадут китель В защиту от боли и зла \nОтыщу это дивное место \nГде из детей вываривают детство \nВ ее кармане будет ключ от всех дверей \nНе разуваясь заберет твоих детей \nВо всем виновен точно чей-то грех, но чей? \nХоть и не просит я иду Хоть и не просит я иду \nХоть и не просит я иду за ней По пути потерпел неудачу \nИ вновь угодил к ней в карман\nГде я был перемешан со сдачей \nВсех тех, кто ее мне давал \nСкорей бы найти это место \nГде из детей вываривают детство \nВ ее кармане будет ключ от всех дверей Не разуваясь заберет твоих детей \nВо всем виновен точно чей-то грех, но чей? \nХоть и не просит я иду \nХоть и не просит я иду \nХоть и не просит я иду за ней \nСостоит из глубоких отметин \nЛихая дорога к котлу \nОт судьбы спустя до

In [5]:
song3

'Я в лесах наберу слова,\nЯ огонь напою вином.\nПод серпом как волна — трава,\nЯ разбавлю надежду сном.\nТебя творить —\nтри года не говорить.\nСердце сварено в молоке,\nЛист крапивы — в глазах костер.\nЛунный свет на твоей руке,\nНа рубашке — красный узор.\nНа рубашке — красный петух,\nА и мне ли жалеть огня?\nКак захватит от дыма дух,\nКак светло улыбнется князь!\nТебя ворожить —\nБосой по углям ходить.\nТебя целовать —\nПод пеплом звезды считать.\nЗа три года траву соткать,\nТемным волосом вышить путь,\nИскры все на него собрать —\nПальцы болью горят, ну и пусть.\nКровь делю на двоих без слов,\nПочернеют снега к весне,\nАлой лентой ночных костров\nСвою душу отдам тебе.\nЗнай, зола —\nВсе слезы выплакала.\nТы тоже знай, смола —\nВсе ветры я прогнала.\nГде теперь взять тепла —\nВсю душу я отдала,\nА другая тебя нашла,\nДругая за руку увела,\nЯ ее за то прокляла.\nБудет время, и будет ночь…\nКак в голодный, беззвездный час\nТы беги, разлучница, прочь —\nОбернется огнем мой князь.\nВспы

## Преобразование данных

Создадим фунуцию `text_to_words`, которая будет 
1) приводить слова к одному регистру;
2) производить токенизацию;
3) производить поиск частей речи;
4) производить лемматизацию.

In [6]:
morph = MorphAnalyzer() # инициализация объекта класса MorphAnalyzer 

def text_to_words(text):
    words = word_tokenize(text.lower()) # токенизация текста, приведение слова к нижнему регистру
    lemmatized_words = [] # создание пустого списка для лемматизированных слов
    
    for word in words:
        if word.isalpha(): # проверка, является ли слово буквенным 
            token = word.strip(string.punctuation) # удаление знаков препинания 
            parsed_word = morph.parse(token)[0] # морфологический анализ слова
            lemma = parsed_word.normal_form # лемматизация слова
            pos = parsed_word.tag.POS # получение части речи и сохранение ее в переменную pos
            lemmatized_words.append((lemma, pos)) # добавление кортежа с леммой и частью речи слова в список lemmatized_words
    
    return lemmatized_words

В результате реализации функции получается список кортежей с леммами слов песни и их частями речи. Применим эту функцию к нашим песням:

In [7]:
song1_words = text_to_words(song1)
song2_words = text_to_words(song2)
song3_words = text_to_words(song3)

In [8]:
song1_words

[('есть', 'INFN'),
 ('такой', 'ADJF'),
 ('дорога', 'NOUN'),
 ('назад', 'ADVB'),
 ('не', 'PRCL'),
 ('вести', 'VERB'),
 ('на', 'PREP'),
 ('чужой', 'ADJF'),
 ('берег', 'NOUN'),
 ('я', 'NPRO'),
 ('прилив', 'NOUN'),
 ('стеречь', 'VERB'),
 ('парус', 'NOUN'),
 ('обмануть', 'GRND'),
 ('ветер', 'NOUN'),
 ('стих', 'NOUN'),
 ('навсегда', 'ADVB'),
 ('плоский', 'ADJF'),
 ('зеркало', 'NOUN'),
 ('стать', 'VERB'),
 ('морской', 'ADJF'),
 ('вода', 'NOUN'),
 ('обернуться', 'INFN'),
 ('бы', 'PRCL'),
 ('лента', 'NOUN'),
 ('в', 'PREP'),
 ('чужой', 'ADJF'),
 ('волос', 'NOUN'),
 ('плыть', 'INFN'),
 ('к', 'PREP'),
 ('ты', 'NPRO'),
 ('до', 'PREP'),
 ('рассвет', 'NOUN'),
 ('не', 'PRCL'),
 ('ведать', 'GRND'),
 ('страх', 'NOUN'),
 ('шёлк', 'NOUN'),
 ('в', 'PREP'),
 ('рука', 'NOUN'),
 ('родный', 'ADJF'),
 ('опуститься', 'INFN'),
 ('легко', 'ADVB'),
 ('вспоминать', 'VERB'),
 ('мой', 'ADJF'),
 ('имя', 'NOUN'),
 ('прикасаться', 'VERB'),
 ('рука', 'NOUN'),
 ('я', 'NPRO'),
 ('по', 'PREP'),
 ('дно', 'NOUN'),
 ('бы', 'PRC

Удалим "стоп-слова":

In [9]:
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['твой', 'это']) # при просмотре результатов очистки текста от "стоп-слов" было обнаружено, 
                                   # что слово "твой" не было приведено к начальной форме и поэтому не удалилось 
                                   # при очистке текста от "стоп-слов" (в списке "стоп-слов" слова "твой" нет), 
                                   # поэтому мы добавляем его в список "стоп-слов", также добавим слово "это",
                                   # которого также нет в списке "стоп-слов"

In [10]:
song1_words = [word for word in song1_words if word[0] not in russian_stopwords]
song1_words

[('дорога', 'NOUN'),
 ('назад', 'ADVB'),
 ('вести', 'VERB'),
 ('чужой', 'ADJF'),
 ('берег', 'NOUN'),
 ('прилив', 'NOUN'),
 ('стеречь', 'VERB'),
 ('парус', 'NOUN'),
 ('обмануть', 'GRND'),
 ('ветер', 'NOUN'),
 ('стих', 'NOUN'),
 ('навсегда', 'ADVB'),
 ('плоский', 'ADJF'),
 ('зеркало', 'NOUN'),
 ('стать', 'VERB'),
 ('морской', 'ADJF'),
 ('вода', 'NOUN'),
 ('обернуться', 'INFN'),
 ('лента', 'NOUN'),
 ('чужой', 'ADJF'),
 ('волос', 'NOUN'),
 ('плыть', 'INFN'),
 ('рассвет', 'NOUN'),
 ('ведать', 'GRND'),
 ('страх', 'NOUN'),
 ('шёлк', 'NOUN'),
 ('рука', 'NOUN'),
 ('родный', 'ADJF'),
 ('опуститься', 'INFN'),
 ('легко', 'ADVB'),
 ('вспоминать', 'VERB'),
 ('имя', 'NOUN'),
 ('прикасаться', 'VERB'),
 ('рука', 'NOUN'),
 ('дно', 'NOUN'),
 ('морской', 'ADJF'),
 ('навстречу', 'ADVB'),
 ('пойти', 'VERB'),
 ('компас', 'NOUN'),
 ('старый', 'ADJF'),
 ('сломаться', 'VERB'),
 ('игла', 'NOUN'),
 ('парус', 'NOUN'),
 ('стереться', 'VERB'),
 ('дыра', 'NOUN'),
 ('палящий', 'ADJF'),
 ('светить', 'VERB'),
 ('ветер',

In [11]:
song2_words = [word for word in song2_words if word[0] not in russian_stopwords]
song2_words

[('судьба', 'NOUN'),
 ('выбирать', 'VERB'),
 ('несмелый', 'ADJF'),
 ('порой', 'ADVB'),
 ('везти', 'VERB'),
 ('прожить', 'INFN'),
 ('размер', 'NOUN'),
 ('пока', 'ADVB'),
 ('всё', 'PRCL'),
 ('большой', 'ADJS'),
 ('вести', 'VERB'),
 ('дивный', 'ADJF'),
 ('место', 'NOUN'),
 ('ребёнок', 'NOUN'),
 ('вываривать', 'VERB'),
 ('детство', 'NOUN'),
 ('говорить', 'VERB'),
 ('храбрый', 'ADJF'),
 ('обитель', 'NOUN'),
 ('сердце', 'NOUN'),
 ('покрыть', 'VERB'),
 ('броня', 'NOUN'),
 ('дойти', 'VERB'),
 ('выдать', 'VERB'),
 ('китель', 'NOUN'),
 ('защита', 'NOUN'),
 ('боль', 'NOUN'),
 ('зло', 'NOUN'),
 ('отыскать', 'VERB'),
 ('дивный', 'ADJF'),
 ('место', 'NOUN'),
 ('ребёнок', 'NOUN'),
 ('вываривать', 'VERB'),
 ('детство', 'NOUN'),
 ('карман', 'NOUN'),
 ('ключ', 'NOUN'),
 ('весь', 'ADJF'),
 ('дверь', 'NOUN'),
 ('разуваться', 'GRND'),
 ('забрать', 'VERB'),
 ('ребёнок', 'NOUN'),
 ('весь', 'ADJF'),
 ('виновный', 'ADJS'),
 ('точно', 'ADVB'),
 ('грех', 'NOUN'),
 ('чей', 'ADJF'),
 ('просить', 'VERB'),
 ('идти',

In [12]:
song3_words = [word for word in song3_words if word[0] not in russian_stopwords]
song3_words

[('лес', 'NOUN'),
 ('набрать', 'VERB'),
 ('слово', 'NOUN'),
 ('огонь', 'NOUN'),
 ('напеть', 'VERB'),
 ('вино', 'NOUN'),
 ('серп', 'NOUN'),
 ('волна', 'NOUN'),
 ('трава', 'NOUN'),
 ('разбавить', 'VERB'),
 ('надежда', 'NOUN'),
 ('сон', 'NOUN'),
 ('творить', 'INFN'),
 ('год', 'NOUN'),
 ('говорить', 'INFN'),
 ('сердце', 'NOUN'),
 ('сварить', 'PRTS'),
 ('молоко', 'NOUN'),
 ('лист', 'NOUN'),
 ('крапива', 'NOUN'),
 ('глаз', 'NOUN'),
 ('костёр', 'NOUN'),
 ('лунный', 'ADJF'),
 ('свет', 'NOUN'),
 ('рука', 'NOUN'),
 ('рубашка', 'NOUN'),
 ('красный', 'ADJF'),
 ('узор', 'NOUN'),
 ('рубашка', 'NOUN'),
 ('красный', 'ADJF'),
 ('петух', 'NOUN'),
 ('жалеть', 'INFN'),
 ('огонь', 'NOUN'),
 ('захватить', 'VERB'),
 ('дым', 'NOUN'),
 ('дух', 'NOUN'),
 ('светло', 'ADVB'),
 ('улыбнуться', 'VERB'),
 ('князь', 'NOUN'),
 ('ворожить', 'INFN'),
 ('босой', 'ADJF'),
 ('уголь', 'NOUN'),
 ('ходить', 'INFN'),
 ('целовать', 'INFN'),
 ('пепел', 'NOUN'),
 ('звезда', 'NOUN'),
 ('считать', 'INFN'),
 ('год', 'NOUN'),
 ('трава

Для произведения тематического моделирования нам понадобятся только леммы, создадим для каждой песни список только с леммами:

In [13]:
song1_lemmas = [word for word, _ in song1_words]
song2_lemmas = [word for word, _ in song2_words]
song3_lemmas = [word for word, _ in song3_words]

In [14]:
song1_lemmas

['дорога',
 'назад',
 'вести',
 'чужой',
 'берег',
 'прилив',
 'стеречь',
 'парус',
 'обмануть',
 'ветер',
 'стих',
 'навсегда',
 'плоский',
 'зеркало',
 'стать',
 'морской',
 'вода',
 'обернуться',
 'лента',
 'чужой',
 'волос',
 'плыть',
 'рассвет',
 'ведать',
 'страх',
 'шёлк',
 'рука',
 'родный',
 'опуститься',
 'легко',
 'вспоминать',
 'имя',
 'прикасаться',
 'рука',
 'дно',
 'морской',
 'навстречу',
 'пойти',
 'компас',
 'старый',
 'сломаться',
 'игла',
 'парус',
 'стереться',
 'дыра',
 'палящий',
 'светить',
 'ветер',
 'попутный',
 'взлюбить',
 'ветер',
 'брат',
 'ветер',
 'осерчать',
 'хоронить',
 'боль',
 'венчать',
 'печаль',
 'бурун',
 'морской',
 'пробежать',
 'нелегко',
 'вспоминать',
 'имя',
 'прикасаться',
 'рука',
 'третий',
 'год',
 'звать',
 'эхо',
 'ответ',
 'обмануть',
 'ветер',
 'запутать',
 'след',
 'сталь',
 'глаз',
 'забыть',
 'грудь',
 'ледяной',
 'морской',
 'вода',
 'обернуться',
 'лента',
 'чужой',
 'волос',
 'плыть',
 'рассвет',
 'ведать',
 'страх',
 'шёлк',

Посмотрим какие слова встречаются чаще всего. Для этого создадим функцию, которая считает частоту для каждого слова и находит самые часто встречающиеся слова.

In [15]:
def most_common_words(word_list, num_words=3):
    word_counts = Counter(word_list)  # создание словаря с подсчетом уникальных значений в списке слов

    most_common_words = word_counts.most_common(num_words) # нахождение самых часто встречающихся слов
    
    for word, count in most_common_words:
        print(f"{word}: {count}")

In [16]:
most_common_words(song1_lemmas, 10)

ветер: 7
рука: 6
морской: 5
вспоминать: 5
имя: 5
прикасаться: 4
чужой: 3
парус: 2
обмануть: 2
вода: 2


In [17]:
most_common_words(song2_lemmas, 10)

ребёнок: 6
просить: 6
идти: 6
весь: 5
место: 4
детство: 4
дивный: 3
вываривать: 3
карман: 3
судьба: 2


In [18]:
most_common_words(song3_lemmas, 10)

огонь: 3
всё: 3
лес: 2
слово: 2
трава: 2
год: 2
костёр: 2
рука: 2
рубашка: 2
красный: 2


Произведем тематическое моделирование. Создадим функцию, в которой для тематического моделирования мы используем модель Latent Dirichlet Allocation (LDA) и для более наглядного представления результатов создаем таблицу. В колонке 'Значимость слов для темы' мы сможем увидеть какой вес имеют слова для каждой выделенной темы.

In [19]:
def lda_table(words_lemmas, num_topics=5): # выберем количество тем равное пяти
    corpus = [[word] for word in words_lemmas]  

    dic = corpora.Dictionary(corpus)
    corpus = [dic.doc2bow(text) for text in corpus]

    lda_model = gensim.models.LdaMulticore(corpus=corpus, id2word=dic, num_topics=num_topics) # обучение модели

    # получение данных из модели LDA для создания таблицы
    topics_data = []
    for topic_id, topic_words in lda_model.print_topics():
        topics_data.append([topic_id, topic_words])

    # создание таблицы
    df = pd.DataFrame(topics_data, columns=['Индекс темы', 'Значимость слов для темы'])

    pd.set_option('display.max_colwidth', None) # расширение колонки 'Значимость слов для темы'
    
    return df

In [20]:
lda_table(song1_lemmas)

,Индекс темы,Значимость слов для темы
0,0,"0.144*""вспоминать"" + 0.061*""пробежать"" + 0.034*""морской"" + 0.034*""чужой"" + 0.034*""родный"" + 0.034*""бурун"" + 0.033*""хоронить"" + 0.033*""лента"" + 0.033*""эхо"" + 0.033*""венчать"""
1,1,"0.110*""морской"" + 0.058*""шёлк"" + 0.058*""опуститься"" + 0.058*""волос"" + 0.032*""прикасаться"" + 0.032*""парус"" + 0.032*""дорога"" + 0.032*""ледяной"" + 0.032*""брат"" + 0.032*""прилив"""
2,2,"0.094*""прикасаться"" + 0.065*""обернуться"" + 0.065*""боль"" + 0.064*""чужой"" + 0.036*""рассвет"" + 0.035*""нелегко"" + 0.035*""венчать"" + 0.035*""брат"" + 0.035*""плыть"" + 0.035*""родный"""
3,3,"0.157*""ветер"" + 0.048*""имя"" + 0.048*""легко"" + 0.048*""осерчать"" + 0.048*""страх"" + 0.048*""печаль"" + 0.048*""ведать"" + 0.026*""рассвет"" + 0.026*""дно"" + 0.026*""хоронить"""
4,4,"0.187*""рука"" + 0.097*""имя"" + 0.066*""обмануть"" + 0.066*""вода"" + 0.036*""лента"" + 0.036*""берег"" + 0.036*""нелегко"" + 0.036*""плыть"" + 0.036*""грудь"" + 0.036*""глаз"""


In [21]:
lda_table(song2_lemmas)

,Индекс темы,Значимость слов для темы
0,0,"0.123*""ребёнок"" + 0.123*""идти"" + 0.099*""весь"" + 0.075*""место"" + 0.052*""судьба"" + 0.029*""карман"" + 0.029*""точно"" + 0.028*""найти"" + 0.028*""размер"" + 0.028*""выдать"""
1,1,"0.072*""чей"" + 0.072*""грех"" + 0.072*""дверь"" + 0.040*""идти"" + 0.040*""просить"" + 0.040*""детство"" + 0.040*""карман"" + 0.040*""точно"" + 0.040*""лихой"" + 0.039*""защита"""
2,2,"0.110*""просить"" + 0.083*""дивный"" + 0.057*""год"" + 0.032*""ребёнок"" + 0.032*""весь"" + 0.032*""место"" + 0.032*""путь"" + 0.031*""забрать"" + 0.031*""пока"" + 0.031*""котёл"""
3,3,"0.112*""детство"" + 0.077*""разуваться"" + 0.042*""забрать"" + 0.042*""храбрый"" + 0.042*""везти"" + 0.042*""вновь"" + 0.042*""дойти"" + 0.042*""скорый"" + 0.042*""потерпеть"" + 0.042*""глубокий"""
4,4,"0.098*""вываривать"" + 0.068*""виновный"" + 0.068*""ключ"" + 0.038*""просить"" + 0.037*""карман"" + 0.037*""путь"" + 0.037*""прожить"" + 0.037*""сердце"" + 0.037*""несмелый"" + 0.037*""выбирать"""


In [22]:
lda_table(song3_lemmas)

,Индекс темы,Значимость слов для темы
0,0,"0.054*""слово"" + 0.030*""князь"" + 0.030*""рука"" + 0.030*""бежать"" + 0.030*""надежда"" + 0.030*""затлеть"" + 0.030*""найти"" + 0.030*""пусть"" + 0.030*""вспыхнуть"" + 0.030*""весна"""
1,1,"0.060*""отдать"" + 0.033*""год"" + 0.033*""кровь"" + 0.033*""снег"" + 0.033*""путь"" + 0.033*""проклясть"" + 0.033*""светло"" + 0.033*""делить"" + 0.033*""петух"" + 0.033*""свет"""
2,2,"0.042*""лес"" + 0.042*""рубашка"" + 0.042*""костёр"" + 0.042*""трава"" + 0.023*""знать"" + 0.023*""сердце"" + 0.023*""разлучница"" + 0.023*""год"" + 0.023*""забирать"" + 0.023*""обернуться"""
3,3,"0.073*""огонь"" + 0.050*""красный"" + 0.028*""знать"" + 0.028*""свой"" + 0.028*""считать"" + 0.028*""искра"" + 0.028*""подол"" + 0.028*""ходить"" + 0.028*""беззвёздный"" + 0.028*""дом"""
4,4,"0.072*""всё"" + 0.049*""весь"" + 0.049*""душа"" + 0.027*""жалеть"" + 0.027*""рука"" + 0.027*""князь"" + 0.027*""крапива"" + 0.027*""лента"" + 0.027*""серп"" + 0.027*""выплакать"""


Произведем поиск устойчивых словосочетаний (n-gramm). Для этого создадим функцию, которая создает биграммы, подсчитывает их частоту и для более наглядного представления результатов создает таблицу с сортировкой частоты по убыванию.

In [23]:
def bigram_table(words):
    bigrams = list(ngrams(words, 2)) # создание биграмм
    bigram_freq = Counter(bigrams) # подсчет частоты биграмм
    
    df = pd.DataFrame(bigram_freq.most_common(), columns=['Пары слов', 'Частота']) # создание таблицы  
                                                                                   # и сортировка по убыванию частоты
    return df

In [24]:
bigram_table(song1_lemmas).head(10)

,Пары слов,Частота
0,"(вспоминать, имя)",5
1,"(имя, прикасаться)",4
2,"(прикасаться, рука)",4
3,"(обмануть, ветер)",2
4,"(морской, вода)",2
5,"(вода, обернуться)",2
6,"(обернуться, лента)",2
7,"(лента, чужой)",2
8,"(чужой, волос)",2
9,"(волос, плыть)",2


In [25]:
bigram_table(song1_lemmas).shape

(78, 2)

In [26]:
bigram_table(song2_lemmas).head(10)

,Пары слов,Частота
0,"(просить, идти)",6
1,"(идти, просить)",4
2,"(дивный, место)",3
3,"(место, ребёнок)",3
4,"(ребёнок, вываривать)",3
5,"(вываривать, детство)",3
6,"(детство, карман)",2
7,"(карман, ключ)",2
8,"(ключ, весь)",2
9,"(весь, дверь)",2


In [27]:
bigram_table(song2_lemmas).shape

(75, 2)

In [28]:
bigram_table(song3_lemmas).head(10)

,Пары слов,Частота
0,"(рубашка, красный)",2
1,"(душа, отдать)",2
2,"(лес, набрать)",1
3,"(набрать, слово)",1
4,"(слово, огонь)",1
5,"(огонь, напеть)",1
6,"(напеть, вино)",1
7,"(вино, серп)",1
8,"(серп, волна)",1
9,"(волна, трава)",1


In [29]:
bigram_table(song3_lemmas).shape

(112, 2)

Посмотрим на устойчивые словосочетания для трех слов:

In [30]:
def bigram_table_three(words):
    bigrams = list(ngrams(words, 3))
    bigram_freq = Counter(bigrams)
    
    df = pd.DataFrame(bigram_freq.most_common(), columns=['Пары слов', 'Частота'])
    
    return df

In [31]:
bigram_table_three(song1_lemmas).head(10)

,Пары слов,Частота
0,"(вспоминать, имя, прикасаться)",4
1,"(имя, прикасаться, рука)",4
2,"(морской, вода, обернуться)",2
3,"(вода, обернуться, лента)",2
4,"(обернуться, лента, чужой)",2
5,"(лента, чужой, волос)",2
6,"(чужой, волос, плыть)",2
7,"(волос, плыть, рассвет)",2
8,"(плыть, рассвет, ведать)",2
9,"(рассвет, ведать, страх)",2


In [32]:
bigram_table_three(song2_lemmas).head(10)

,Пары слов,Частота
0,"(просить, идти, просить)",4
1,"(идти, просить, идти)",4
2,"(место, ребёнок, вываривать)",3
3,"(ребёнок, вываривать, детство)",3
4,"(дивный, место, ребёнок)",2
5,"(вываривать, детство, карман)",2
6,"(детство, карман, ключ)",2
7,"(карман, ключ, весь)",2
8,"(ключ, весь, дверь)",2
9,"(весь, дверь, разуваться)",2


In [33]:
bigram_table_three(song3_lemmas).head(10)

,Пары слов,Частота
0,"(лес, набрать, слово)",1
1,"(набрать, слово, огонь)",1
2,"(слово, огонь, напеть)",1
3,"(огонь, напеть, вино)",1
4,"(напеть, вино, серп)",1
5,"(вино, серп, волна)",1
6,"(серп, волна, трава)",1
7,"(волна, трава, разбавить)",1
8,"(трава, разбавить, надежда)",1
9,"(разбавить, надежда, сон)",1


Слова в двух наиболее часто встречающихся сочетаниях в первой песне пересекаются: (вспоминать, имя, прикасаться) и (имя, прикасаться, рука). Во второй песне такое пересечение еще более явное, просто меняется порядок слов: (просить, идти, просить)	и (идти, просить, идти). Это может быть одним сочтением из четырех слов. Проверим это, создав аналогичную функцию для четырех слов.

In [34]:
def bigram_table_four(words):
    bigrams = list(ngrams(words, 4))
    bigram_freq = Counter(bigrams)
    
    df = pd.DataFrame(bigram_freq.most_common(), columns=['Пары слов', 'Частота'])
    
    return df

In [35]:
bigram_table_four(song1_lemmas).head(10)

,Пары слов,Частота
0,"(вспоминать, имя, прикасаться, рука)",4
1,"(морской, вода, обернуться, лента)",2
2,"(вода, обернуться, лента, чужой)",2
3,"(обернуться, лента, чужой, волос)",2
4,"(лента, чужой, волос, плыть)",2
5,"(чужой, волос, плыть, рассвет)",2
6,"(волос, плыть, рассвет, ведать)",2
7,"(плыть, рассвет, ведать, страх)",2
8,"(рассвет, ведать, страх, шёлк)",2
9,"(ведать, страх, шёлк, рука)",2


In [36]:
bigram_table_four(song2_lemmas).head(10)

,Пары слов,Частота
0,"(просить, идти, просить, идти)",4
1,"(место, ребёнок, вываривать, детство)",3
2,"(дивный, место, ребёнок, вываривать)",2
3,"(ребёнок, вываривать, детство, карман)",2
4,"(вываривать, детство, карман, ключ)",2
5,"(детство, карман, ключ, весь)",2
6,"(карман, ключ, весь, дверь)",2
7,"(ключ, весь, дверь, разуваться)",2
8,"(весь, дверь, разуваться, забрать)",2
9,"(дверь, разуваться, забрать, ребёнок)",2


Как мы и предполагали, это оказалось сочетания из четырех слов, которые встречаются в первой и второй песнях по 4 раза.

## Рассчет метрики TF-IDF
Показатель TD-IDF — это индикатор того, насколько релевантно слово в контексте документа. Этот показатель возрастает пропорционально количеству раз, когда слово встречается в тексте, и уменьшается пропорционально количеству слов во всех текстах в целом.

* Коэффициент будет выше, если слово характерно именно для этого текста, то есть встречается в данном тексте часто, но не встречается в других текстах.
* Коэффициент будет ниже, если слово не встречается почти нигде или встречается одинаковое количество раз во всех текстах, то есть не характеризует никакой текст в отдельности.

Произведем кодирование данных помощью методологии TF-IDF:

In [37]:
tfidf = TfidfVectorizer() # инициализация объекта TfidfVectorizer
tfidf_matrix = tfidf.fit_transform([word for word in song1_lemmas])
tfidf_values = tfidf.transform([word for word in song1_lemmas]).toarray()

max_tfidf_value = np.max(tfidf_values)
max_tfidf_indices = np.where(tfidf_values == max_tfidf_value)[1]
words_with_max_tfidf = [tfidf.get_feature_names_out()[index] for index in max_tfidf_indices]

print("Максимальное значение TF-IDF для песни №1:", round(max_tfidf_value, 2))
print("Слова с максимальным TF-IDF коэффициентом в песне №1:", words_with_max_tfidf)

Максимальное значение TF-IDF для песни №1: 1.0
Слова с максимальным TF-IDF коэффициентом в песне №1: ['дорога', 'назад', 'вести', 'чужой', 'берег', 'прилив', 'стеречь', 'парус', 'обмануть', 'ветер', 'стих', 'навсегда', 'плоский', 'зеркало', 'стать', 'морской', 'вода', 'обернуться', 'лента', 'чужой', 'волос', 'плыть', 'рассвет', 'ведать', 'страх', 'шёлк', 'рука', 'родный', 'опуститься', 'легко', 'вспоминать', 'имя', 'прикасаться', 'рука', 'дно', 'морской', 'навстречу', 'пойти', 'компас', 'старый', 'сломаться', 'игла', 'парус', 'стереться', 'дыра', 'палящий', 'светить', 'ветер', 'попутный', 'взлюбить', 'ветер', 'брат', 'ветер', 'осерчать', 'хоронить', 'боль', 'венчать', 'печаль', 'бурун', 'морской', 'пробежать', 'нелегко', 'вспоминать', 'имя', 'прикасаться', 'рука', 'третий', 'год', 'звать', 'эхо', 'ответ', 'обмануть', 'ветер', 'запутать', 'след', 'сталь', 'глаз', 'забыть', 'грудь', 'ледяной', 'морской', 'вода', 'обернуться', 'лента', 'чужой', 'волос', 'плыть', 'рассвет', 'ведать', 'стра

Эти слова имеют наибольшую значимость среди всех слов в контексте этой песни.

In [38]:
tfidf = TfidfVectorizer() # инициализация объекта TfidfVectorizer
tfidf_matrix = tfidf.fit_transform([word for word in song2_lemmas])
tfidf_values = tfidf.transform([word for word in song2_lemmas]).toarray()

max_tfidf_value = np.max(tfidf_values)
max_tfidf_indices = np.where(tfidf_values == max_tfidf_value)[1]
words_with_max_tfidf = [tfidf.get_feature_names_out()[index] for index in max_tfidf_indices]

print("Максимальное значение TF-IDF для песни №2:", round(max_tfidf_value, 2))
print("Слова с максимальным TF-IDF коэффициентом в песне №2:", words_with_max_tfidf)

Максимальное значение TF-IDF для песни №2: 1.0
Слова с максимальным TF-IDF коэффициентом в песне №2: ['судьба', 'выбирать', 'несмелый', 'порой', 'везти', 'прожить', 'размер', 'пока', 'всё', 'большой', 'вести', 'дивный', 'место', 'ребёнок', 'вываривать', 'детство', 'говорить', 'храбрый', 'обитель', 'сердце', 'покрыть', 'броня', 'дойти', 'выдать', 'китель', 'защита', 'боль', 'зло', 'отыскать', 'дивный', 'место', 'ребёнок', 'вываривать', 'детство', 'карман', 'ключ', 'весь', 'дверь', 'разуваться', 'забрать', 'ребёнок', 'весь', 'виновный', 'точно', 'грех', 'чей', 'просить', 'идти', 'просить', 'идти', 'просить', 'идти', 'путь', 'потерпеть', 'неудача', 'вновь', 'угодить', 'карман', 'перемешать', 'сдача', 'весь', 'давать', 'скорый', 'найти', 'место', 'ребёнок', 'вываривать', 'детство', 'карман', 'ключ', 'весь', 'дверь', 'разуваться', 'забрать', 'ребёнок', 'весь', 'виновный', 'точно', 'грех', 'чей', 'просить', 'идти', 'просить', 'идти', 'просить', 'идти', 'состоять', 'глубокий', 'отметина', 'ли

In [39]:
tfidf = TfidfVectorizer() # инициализация объекта TfidfVectorizer
tfidf_matrix = tfidf.fit_transform([word for word in song3_lemmas])
tfidf_values = tfidf.transform([word for word in song3_lemmas]).toarray()

max_tfidf_value = np.max(tfidf_values)
max_tfidf_indices = np.where(tfidf_values == max_tfidf_value)[1]
words_with_max_tfidf = [tfidf.get_feature_names_out()[index] for index in max_tfidf_indices]

print("Максимальное значение TF-IDF для песни №3:", round(max_tfidf_value, 2))
print("Слова с максимальным TF-IDF коэффициентом в песне №3:", words_with_max_tfidf)

Максимальное значение TF-IDF для песни №3: 1.0
Слова с максимальным TF-IDF коэффициентом в песне №3: ['лес', 'набрать', 'слово', 'огонь', 'напеть', 'вино', 'серп', 'волна', 'трава', 'разбавить', 'надежда', 'сон', 'творить', 'год', 'говорить', 'сердце', 'сварить', 'молоко', 'лист', 'крапива', 'глаз', 'костёр', 'лунный', 'свет', 'рука', 'рубашка', 'красный', 'узор', 'рубашка', 'красный', 'петух', 'жалеть', 'огонь', 'захватить', 'дым', 'дух', 'светло', 'улыбнуться', 'князь', 'ворожить', 'босой', 'уголь', 'ходить', 'целовать', 'пепел', 'звезда', 'считать', 'год', 'трава', 'соткать', 'тёмный', 'волос', 'вышить', 'путь', 'искра', 'всё', 'собрать', 'палец', 'боль', 'гореть', 'пусть', 'кровь', 'делить', 'двое', 'слово', 'почернеть', 'снег', 'весна', 'ала', 'лента', 'ночной', 'костёр', 'свой', 'душа', 'отдать', 'знать', 'зола', 'всё', 'слеза', 'выплакать', 'знать', 'смола', 'всё', 'ветер', 'прогнать', 'взять', 'тепло', 'весь', 'душа', 'отдать', 'найти', 'рука', 'увести', 'проклясть', 'время', '

Произведем поиск близких по смыслу слов с помощью векторной модели word2vec. Для начала посмотрим на значения для одного слова, например для слова "место":

In [40]:
sentences = [[word for word, pos in words] for words in [song1_words, song2_words, song3_words]] # подготовка данных 
                                                                                                 # для обучения Word2Vec

model = Word2Vec(sentences, window=5, min_count=1, workers=4) # обучение модели Word2Vec

similar_words = model.wv.most_similar("место", topn=3) # поиск трех наиболее близких по смыслу слов к слову "место"
print(similar_words)

[('взять', 0.318543016910553), ('родный', 0.3125579357147217), ('звать', 0.18941907584667206)]


Выведем близкие по смыслу слова:

In [41]:
all_sentences = [word for sentence in sentences for word in sentence]
model = Word2Vec(sentences, window=5, min_count=1, workers=4)

similar_words_dic = {}

for word in model.wv.key_to_index.keys():
    similar_words = model.wv.most_similar(word, topn=3)  # поиск трех близких по смыслу слов
    similar_words_dic[word] = [w for w, _ in similar_words]

for word, similar_words in similar_words_dic.items():
    print(f"Близкие по смыслу слова для слова '{word}': {similar_words}")

Близкие по смыслу слова для слова 'ветер': ['ответ', 'выплакать', 'стать']
Близкие по смыслу слова для слова 'рука': ['звезда', 'опуститься', 'чужой']
Близкие по смыслу слова для слова 'весь': ['взять', 'хоронить', 'дым']
Близкие по смыслу слова для слова 'просить': ['сон', 'улыбнуться', 'снег']
Близкие по смыслу слова для слова 'идти': ['везти', 'боль', 'покрыть']
Близкие по смыслу слова для слова 'ребёнок': ['большой', 'взять', 'прожить']
Близкие по смыслу слова для слова 'год': ['трава', 'легко', 'спустя']
Близкие по смыслу слова для слова 'морской': ['лента', 'тепло', 'спустя']
Близкие по смыслу слова для слова 'вспоминать': ['опуститься', 'точно', 'эхо']
Близкие по смыслу слова для слова 'имя': ['смола', 'стать', 'забрать']
Близкие по смыслу слова для слова 'детство': ['слово', 'дно', 'улыбнуться']
Близкие по смыслу слова для слова 'место': ['взять', 'родный', 'звать']
Близкие по смыслу слова для слова 'всё': ['трава', 'красный', 'выплакать']
Близкие по смыслу слова для слова 'бол

Предполагается, что близость по смыслу между словами определяется сходством распределения контекстов, в которых они встречаются. Однако близкие по смыслу слова у нас получились неочевидными. Скорее всего это обусловлено спецификой текста песен на которых обучалась модель (например тем, что текст песен не такой длинный).

## Вывод по полученным преобразованным данным

Чаще всего в первой песне встречаются слова "ветер" (встречается 7 раз), "рука" (встречается 6 раз), "морской", "вспоминать" и "имя" (встречается по 5 раз), во второй песне - слова "ребёнок", "просить" и "идти" (встречаются по 6 раз), "весь" (встречается 5 раз), в третьей песне - слова "огонь" и "всё" (встречаются по 3 раза).

Чаще всего в первой песне встречаются сочетания слов "вспоминать" и "имя" (встречается 5 раз), "имя" и "прикасаться"(встречается 4 раза), "прикасаться" и "рука"(встречается 4 раза), во второй - сочетания слов "просить" и "идти" (встречается 6 раза, в другом порядке: "идти" и "просить" встречается 4 раза), в третьей - сочетания слов "рубашка" и "красный" (встречается 2 раза), "душа" и "отдать" (встречается 2 раза). Скорее всего такая частота объясняется тем, что сочетания этих слов встречаются в припеве песен, который повторяется несколько раз.  

Среди сочетаний из четырех слов в первой песне чаще всего встречаются сочетания слов "вспоминать", "имя",  "прикасаться" и "рука" (встречаются по 4 раза), во второй песне - сочетания слов "просить", "идти", "просить" и "идти" (встречаются по 4 раза), в третьей песне нет сочетаний из четрех (и из трех) слов, которые встречались бы чаще всего.

Слов и сочетаний слов, которые встречаются редко (один раз) во всех песнях достаточно много.